In [1]:
import numpy as np
from src.synthetic_data import ex4_sample
from src.superlearner import SuperLearner
import sklearn
from statsmodels.api import GLM,families

In [2]:
seed = 1
np.random.seed(seed)
n = 10000
d = 5
k = 5
data = ex4_sample(n,d)
outcome_learners = [sklearn.linear_model.LogisticRegression(),sklearn.neighbors.KNeighborsClassifier()]
pt_learners = [sklearn.linear_model.LogisticRegression(),sklearn.neighbors.KNeighborsClassifier()]

Step 1: Estimate $\bar{Q}_{0} = \mathbb{E}[Y\mid T, X]$ 

In [3]:
trt = data['t']
y = data['y']
x = data['x']
x_outcome = np.column_stack((x,trt))

outcome_model = SuperLearner(k,seed,outcome_learners)
outcome_model.train_binary(x_outcome,y)

Q00 = outcome_model.predict_binary(np.column_stack((x,np.zeros_like(trt))))
Q01 = outcome_model.predict_binary(np.column_stack((x,np.ones_like(trt))))
Q0 = outcome_model.predict_binary(x_outcome)

Step 2: Estimate the propensity score $g_{0}=P_{0}(T\mid X)$

In [4]:
treatment_model = SuperLearner(k,seed,pt_learners)
treatment_model.train_binary(x,trt)

g = treatment_model.predict_binary(x)

Step 3: Calculate the clever covariate $H(T,X) = I(T=1)/g_{n}(1\mid X) - I(T=0)/g_{n}(0\mid X)$ and run logistic regression of the outcome $Y$ on the clever covariate using $logit\ \bar{Q}_{n}^{0}(T,X)$, as the offset

In [5]:
HT = trt/g - (1-trt)/(1-g)
H1 = 1/g
H0 = -1/(1-g)
logit_Q0 = np.log(Q0/(1-Q0))
model = GLM(y, HT, family=families.Binomial(), offset=logit_Q0).fit()
eps = float(model.params[0])
print(eps)


-0.00043856077661477873


Step 4: Update the outcome using the fitted parametric working model, $logit\ \bar{Q}_{n}^{1}(T,X) = logit\ \bar{Q}_{n}^{0}(T,X) + \epsilon H(T,X)$

In [6]:
logit_Q00 = np.log(Q00/(1-Q00))
logit_Q01 = np.log(Q01/(1-Q01))
logit_Q10 = logit_Q00 + eps*H0
logit_Q11 = logit_Q01 + eps*H1

Q10 = 1/(1+np.exp(-logit_Q10))
Q11 = 1/(1+np.exp(-logit_Q11))
Q1T = trt*Q11 + (1-trt)*Q10

Step 5: Calculate ATE as $\frac{1}{n}\sum_{i=1}^{n} [\bar{Q}_{n}^{1}(1,X_{i}) - \bar{Q}_{n}^{1}(0,X_{i})]$

In [7]:
ATE = np.mean(Q11) - np.mean(Q10)
# Influence curve
IC = HT*(y - Q1T) + Q11 - Q10 - ATE
ATE_var = np.mean((IC - np.mean(IC))**2)
ATE_sd = np.sqrt(ATE_var/n)

print(f"ATE (95% CI)= {ATE:.4f} ({ATE-1.96*ATE_sd:.4f},{ATE+1.96*ATE_sd:.4f})")

ATE (95% CI)= 0.0053 (-0.0142,0.0248)
